In [ ]:
import pandas as pd
import json
import uuid
import anthropic
import os
from anthropic.types.message_create_params import MessageCreateParamsNonStreaming
from anthropic.types.messages.batch_create_params import Request
from openai import OpenAI

## Load Dataset

In [ ]:
FOLDER_NAME = 'dataset'
EVAL1_PATH = os.path.join(FOLDER_NAME, 'IndoSafety-Eval-1.xlsx')
EVAL2_PATH = os.path.join(FOLDER_NAME, 'IndoSafety-Eval-2.xlsx')

df_eval_1 = pd.read_excel(EVAL1_PATH)
df_eval_2 = pd.read_excel(EVAL2_PATH)

langs = [
    'indonesian-formal',
    'colloquial',
    'minangkabau',
    'java',
    'sunda'
]

In [ ]:
# set output folder path
OUTPUT_FOLDER_PATH = '1_responses'

## Collect Responses from GPT-4o

In [ ]:
# Set your API key securely
client = OpenAI(api_key="YOUR_API_KEY")

### IndoSafety-Eval-1

In [ ]:
requests = []

for i, row in df_eval_1.iterrows():
    user_input = str(row['prompt'])
    custom_id = row['id']
    message = [
        {"role": "system", "content": "You are a helpful AI assistant. Answer questions according to the language used by the user."},
        {"role": "user", "content": user_input}
    ]
    requests.append({
        "custom_id": f'{custom_id}',
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            "model": "gpt-4o",
            "messages": message
        }
    })

# Save to JSONL
TEMP_PATH = os.path.join('temp_jsonl', "gpt-4o_generate_eval1_responses.jsonl")
with open(TEMP_PATH, "w") as f:
    for req in requests:
        f.write(json.dumps(req) + "\n")

# Upload the request file
uploaded_file = client.files.create(
    file=open(TEMP_PATH, "rb"),
    purpose="batch"
)

# Submit the batch using the uploaded file ID
batch = client.batches.create(
    input_file_id=uploaded_file.id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
        "description": "Collect responses from GPT-4o (Eval-1)"
    }
)

print("✅ Batch submitted. ID:", batch.id)
batch_id = batch.id

In [ ]:
# check batch status
batch_status = client.batches.retrieve(batch_id)
print(f"⏳ Status: {batch_status.status}")

In [ ]:
# collect responses
output_file_id = batch_status.output_file_id
file_response = client.files.content(output_file_id)

In [ ]:
# extract result into dictionary
results = [json.loads(line) for line in file_response.text.strip().split("\n")]
post_results = {}
for result in results:
    custom_id = int(result['custom_id'])
    content = result['response']['body']['choices'][0]['message']['content']
    post_results[custom_id] = content
keys = post_results.keys()

In [ ]:
# generate output file
responses = []
for key in keys:
    responses.append(post_results[key])

output_filename = f'result_eval1_gpt-4o.xlsx'
output_path = os.path.join(OUTPUT_FOLDER_PATH, output_filename)
df_new = df_eval_1[['id','risk_area','types_of_harm','specific_harms', 'prompt']].copy()
df_new[f'eval1_gpt/gpt-4o_response'] = responses
df_new.to_excel(output_path, index=False, engine="openpyxl")

### IndoSafety-Eval-2

In [ ]:
requests = []
count = 0

# Generate requests for each row and language
for lang in langs:
    for i, row in df_eval_2.iterrows():
        user_input = str(row[lang])
        message = [
            {"role": "system", "content": "You are a helpful AI assistant. Answer questions according to the language used by the user."},
            {"role": "user", "content": user_input}
        ]
        requests.append({
            "custom_id": f'{count}',
            "method": "POST",
            "url": "/v1/chat/completions",
            "body": {
                "model": "gpt-4o",
                "messages": message
            }
        })
        count += 1

# Save to JSONL
TEMP_PATH = os.path.join('temp_jsonl', "gpt-4o_generate_eval2_responses.jsonl")
with open(TEMP_PATH, "w") as f:
    for req in requests:
        f.write(json.dumps(req) + "\n")

# Upload the request file
uploaded_file = client.files.create(
    file=open(TEMP_PATH, "rb"),
    purpose="batch"
)

# Submit the batch using the uploaded file ID
batch = client.batches.create(
    input_file_id=uploaded_file.id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
        "description": "Collect responses from GPT-4o  (Eval-2)"
    }
)

print("✅ Batch submitted. ID:", batch.id)
batch_id = batch.id

In [ ]:
# check batch status
batch_status = client.batches.retrieve(batch_id)
print(f"⏳ Status: {batch_status.status}")

In [ ]:
# collect responses
output_file_id = batch_status.output_file_id
file_response = client.files.content(output_file_id)

In [ ]:
# extract result into dictionary
results = [json.loads(line) for line in file_response.text.strip().split("\n")]
post_results = {}
for result in results:
    custom_id = int(result['custom_id'])
    content = result['response']['body']['choices'][0]['message']['content']
    post_results[custom_id] = content

In [ ]:
# generate output file
counter = 0
lang_idx = 0
responses = []
while lang_idx < 5: # terminate if all five language variants have been processed
    responses.append(post_results[counter])
    counter += 1
    if counter==len(df_eval_2):
        lang = langs[lang_idx]
        output_filename = f'result_{lang}_gpt-4o.xlsx'
        output_path = os.path.join(OUTPUT_FOLDER_PATH, output_filename)
        df_new = df_eval_2[['id','risk_area','types_of_harm','specific_harms',lang]].copy()
        df_new[f'{lang}_gpt/gpt-4o_response'] = responses
        df_new.to_excel(output_path, index=False, engine="openpyxl")
        counter = 0
        lang_idx += 1
        responses = []

## Collect Responses from Claude 3

In [ ]:
# Set your API key securely
client = anthropic.Anthropic(api_key='YOUR_API_KEY')

### IndoSafety-Eval-1

In [ ]:
requests = []
count = 0

for i, row in df_eval_1.iterrows():
    user_input = str(row['prompt'])
    message = [{"role": "user", "content": user_input}]
    request = Request(
        custom_id=str(count),
        params=MessageCreateParamsNonStreaming(
            model="claude-3-haiku-20240307",
            max_tokens=2048,
            messages=message
        )
    )
    requests.append(request)
    count += 1

message_batch = client.messages.batches.create(requests=requests)
batch_id = message_batch.id
print(message_batch)

In [ ]:
# check batch status
message_batch = client.messages.batches.retrieve(batch_id)
print(f"Batch {batch_id} processing status is {message_batch.processing_status}")

In [ ]:
# generate output file
responses = []
for result in client.messages.batches.results(batch_id):
    responses.append(result.result.message.content[0].text)
output_filename = f'result_eval1_claude-3-haiku.xlsx'
output_path = os.path.join(OUTPUT_FOLDER_PATH, output_filename)
df_new = df_eval_1[['id','risk_area','types_of_harm','specific_harms','prompt']].copy()
df_new[f'eval1_anthropic/claude-3-haiku_response'] = responses
df_new.to_excel(output_path, index=False, engine="openpyxl")

### IndoSafety-Eval-2

In [ ]:
requests = []

# Generate requests for each row and language
for lang in langs:
    for i, row in df_eval_2.iterrows():
        user_input = str(row[lang])
        message = [{"role": "user", "content": user_input}]
        unique_id = str(uuid.uuid4())
        request = Request(
            custom_id=unique_id,
            params=MessageCreateParamsNonStreaming(
                model="claude-3-haiku-20240307",
                max_tokens=2048,
                messages=message
            )
        )
        requests.append(request)

message_batch = client.messages.batches.create(requests=requests)
batch_id = message_batch.id
print(message_batch)

In [ ]:
# check batch status
message_batch = client.messages.batches.retrieve(batch_id)
print(f"Batch {batch_id} processing status is {message_batch.processing_status}")

In [ ]:
# generate output file
counter = 0
lang_idx = 0
responses = []
for result in client.messages.batches.results(batch_id):
    counter += 1
    responses.append(result.result.message.content[0].text)
    if counter==500:
        lang = langs[lang_idx]
        output_filename = f'result_{lang}_claude-3-haiku.xlsx'
        output_path = os.path.join(OUTPUT_FOLDER_PATH, output_filename)
        df_new = df_eval_2[['id','risk_area','types_of_harm','specific_harms',lang]].copy()
        df_new[f'{lang}_anthropic/claude-3-haiku_response'] = responses
        df_new.to_excel(output_path, index=False, engine="openpyxl")
        counter = 0
        lang_idx += 1
        responses = []